# OOP


In [29]:
class Account:
    '''
    a simple bank account
    ''' #documentation string

    owner:str
    balance:float

    def __init__(self, owner, balance):
        self.owner=owner
        self.balance=balance
    def __repr__(self):
        #return f'Account({self.owner!r}, {self.balance!r})' #hardcoded
        return f'{type(self).__name__}({self.owner!r}, {self.balance!r})'
    def deposit(self, amount):
        self.balance+=amount
    def  withdraw(self, amount):
        self.balance-=amount
    def inquiry(self):
        return self.balance

a=Account("ram", 10.0)
print(a.inquiry)
print(a.owner)
print(vars(a))
print(type(a), type(a).deposit, type(a).inquiry)

<bound method Account.inquiry of Account('ram', 10.0)>
ram
{'owner': 'ram', 'balance': 10.0}
<class '__main__.Account'> <function Account.deposit at 0x7e1a8f40e200> <function Account.inquiry at 0x7e1a8f40e340>


Everything in Python is a dynamic process with very few restriction. If you want to add a new attribute to an object after its been created, you're free to do that.


In [5]:
a=Account("hari", 20.0)
a.creation_date='2020/2/12'
print(a.creation_date)

2020/2/12


In [6]:
del a.creation_date
print(a.creation_date)

AttributeError: 'Account' object has no attribute 'creation_date'

In [10]:
b=Account("shyam", 30.0)
print(getattr(b, 'owner' ))
setattr(a, 'balance', 90.0)
print(getattr(a, 'balance'))
setattr(a, 'Fraud', True)
print(hasattr(a, 'Fraud'))
getattr(a, 'withdraw')(20)
print(getattr(a, 'balance'))

shyam
90.0
True
70.0


In [14]:
getattr(b, 'balance', 'unknown')
# getattr(a, 'creation_date', 'unknown')

30.0

In [15]:
getattr(a, 'creation_date', 'unknown')


'unknown'

In [17]:
w=b.withdraw
print(w)

<bound method Account.withdraw of Account('shyam', 30.0)>


In [19]:
d=b.deposit
print(d(100.0))
print(b.inquiry())

None
130.0


### Operator overloading

In [21]:
class AccountPortfolio:
    def __init__(self):
        self.accounts=[]
    def add_account(self, account):
        self.accounts.append(account)
    def total_funds(self):
        return sum(account.inquiry() for account in self)
    def __len__(self):
        return len(self.accounts)
    def __getitem__(self, index):
        return self.accounts[index]
    def __iter__(self):
        return iter(self.accounts)

port=AccountPortfolio()
port.add_account(Account("Sita", 1000.0))
port.add_account(Account("Sam", 500.0))
print(port.total_funds())
print(len(port))
for account in port:
    print(account)
print(port[1].inquiry())

1500.0
2
Account('Sita', 1000.0)
Account('Sam', 500.0)
500.0


### Inheritance 
It is the mechanism for creating anew class tha specializes of modifies the behaviour of an existing class.  A derived class may redefine any of the attributes inherited and add new attributes of its own.
Inheritance is specified with a comma-separated list of base-class in the calss statement. With no specified base class, a class implicityly inherits from object. 

In [22]:
class MyAccount(Account):
    def panic(self):
        self.withdraw(self.balance)
a=MyAccount('Laxmi', 1000.00)
a.withdraw(10)
print(a.balance)
a.panic()
print(a.balance)

990.0
0.0


In [25]:
import random
class EvilAccount(Account):
    def inquiry(self):
        if random.randint(0, 4)==1:
            return self.balance*1.1
        else:
            return self.balance
a=EvilAccount("Hariom", 100.0)
a.deposit(1000.0)
print(a.inquiry())

1100.0


A derive class would reimplement a method but may also need to call the original implementation. A method can explicitly call the original method using super()

In [26]:
class EvilAccount(Account):
    def inquiry(self):
        if random.randint(0,4)==1:
            return 1.1*super().inquiry()
        else:
            return super().inquiry()
print(a.inquiry())

1100.0


Inheritance might also be used to add additional attributes to instances 

In [30]:
class EvilAccount(Account):
    def __init__(self, owner, balance, factor):
        super().__init__(owner, balance)
        self.factor=factor
    def inquiry(self):
        if random.randint(0,4)==1:
            return self.factor*super().inquiry()
        else:
            return super().inquiry()

# when __init__() is redefined, it is the responsibility of the child to initialize its parent using super().__init__(). Since initializtion of child requires additonal arguments, those still must be passed to the child __init__() method

a= EvilAccount("eva", 1220.09,1)
print(a)

EvilAccount('eva', 1220.09)


Avoid the hardcoding of class names

In [31]:
print(type(a))
print(isinstance(a, Account))

<class '__main__.EvilAccount'>
True


Inheritance establishes a relationship in the type system where any child class will type-check as the parent class. this is 'is a ' relationship. This relationship is used to define object type ontologies or taxonomies. 

### Avoiding Inheritance via composition
Implementation inheritance should be tricky as features that aren't pertinent to the problem being solved may also be inherited. So, composition is used. For eg, instead of inheriting a list in stack, you can buld a stack as an independent class that happens to have a list contained in it. This way there will be no extraneous list methods or nonstack features. You can even modify it to accpet an internal list class as an optional argument which will promote loose coupling of components. this makes it so that instead of hardwiring Stack to depend on list, you can make it depend on any container a user decides to pass in, provided it implements the required interface. this is called *dependency injection*

In [32]:
class Stack(list):
    def push(self, item):
        self.append(item)

s=Stack()
s.push(1)
s.push(2)
print(s.pop())

2


In [33]:
class Stack:
    def __init__(self):
        self._items=list()
    def push(self, item):
        self._items.append(item)
    def pop(self):
        return self._items.pop()
    def __len__(self):
        return len(self._items)

st=Stack()
st.push(2)
st.push(3)
print(st.pop())

3


In [34]:
class Stack:
    def __init__(self, *, container=None):
        if container is None:
            container=list()
        self._items=container
    def push(self, item):
        self._items.append(item)
    def pop(self):
        return self._items.pop()
    def __len__(self):
        return len(self._items)

import array
s=Stack(container=array.array('i'))
s.push(12)
s.push(14)
print(s.pop())

14


In [36]:
class Stack:
    def __init__(self):
        self._items=None
        self._size=0
    def push(self, item):
        self._items=(item, self._items)
        self._size+=1
    def pop(self):
        (item, self._items)=self._items
        self._size-=1
        return item
    def __len__(self):
        return self._size
s=Stack()
s.push(19)
s.push(18)
print(s.pop())

18


### avoiding inheritance via functions
If you're writing a lot of single-method classes, consider using functions instead.  

### Dynamic binding and duck typing
Dynamic binding is runtime mechanism that Python uses to find the attributes of objects, which allows Python to work with instances without regard for their type. attribute binding is independent of what kind of object obj is. If you make a lookup such as obj.name, it will work on any obj whatsoever that happens to have a name attribute. This behaviour is called uck typing.  

### Dangers of inheriting from built-in types
Python's built-in types are implemented in C, so though inheritance is allowed, there is danger of the redefined method not being used.
The collections module has special classes UserDict, UserList and UserString that can be used to make safe subclasses. Try avoiding subclassing a built-in type wherever possible. 

In [37]:
class udict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key.upper(), value)
u=udict()
u['name']='Ram'
u['number']=2
print(u)


{'NAME': 'Ram', 'NUMBER': 2}


In [40]:
u=udict(name='Ram', number=37)
print(u)
u.update(color='blue')
print(u)

{'name': 'Ram', 'number': 37}
{'name': 'Ram', 'number': 37, 'color': 'blue'}


In [1]:
from collections import UserDict

class udict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key.upper(), value)
u=udict(name='Ram', number=37)
print(u)
u.update(color='blue')
print(u)

{'NAME': 'Ram', 'NUMBER': 37}
{'NAME': 'Ram', 'NUMBER': 37, 'COLOR': 'blue'}


### class variables and mehtods. 
A class itself is also an object that can carry state and be manipulated as well. 
 

In [7]:
class Account:
    '''
    a simple bank account
    ''' #documentation string

    owner:str
    balance:float
    num_accounts=0 #class variable

    def __init__(self, owner, balance):
        self.owner=owner
        self.balance=balance
        Account.num_accounts+=1
    def __repr__(self):
        #return f'Account({self.owner!r}, {self.balance!r})' #hardcoded
        return f'{type(self).__name__}({self.owner!r}, {self.balance!r})'
    def deposit(self, amount):
        self.balance+=amount
    def  withdraw(self, amount):
        self.deposit(-amount)
    def inquiry(self):
        return self.balance

a=Account('niva', 100)
b=Account('eva', 2020)
print(Account.num_accounts)
c=a.num_accounts
print(c)

2
2


A class method is a method applied to the class itself, not to instances. A common use is to define alternate instance constructors. The first argument of a class method is always the class itself and by convention, the argument is named cls. 

In [1]:
class Account:
    def __init__(self, owner, balance):
        self.owner=owner
        self.balance=balance
    @classmethod
    def from_xml(cls, data):
        from xml.etree.ElementTree import XML
        doc=XML(data)
        return cls(doc.findtext('owner'), float(doc.findtext('amount')))

data='''
<account>
<owner>Guido</owner>
<amount>1000.0</amount>
</account>
'''
a=Account.from_xml(data)


Class variables and class methods are sometimes used together to configure and control how instances operate. 

In [5]:
import time
class Date:
    datefmt='{year}-{month:02d}-{day:02d}'
    def __init__(self, year, month, day):
        self.year=year
        self.month=month
        self.day=day
    def __str__(self):
        return self.datefmt.format(year=self.year, month=self.month, day=self.day)

    @classmethod
    def from_timestamp(cls, ts):
        tm=time.localtime(ts)
        return cls(tm.tm_year, tm.tm_mon, tm.tm_mday)
    @classmethod
    def today(cls):
        return cls.from_timestamp(time.time())

class MDYDate(Date):
        datefmt='{month}/{day}/{year}'

class DMYDate(Date):
        datefmt='{day}/{month}/{year}'


a=Date(1967, 4,9)
print(a)
b=MDYDate(1967, 4,9)
print(b)
c=DMYDate(1967,4,9)
print(c)

d=MDYDate.today()
e=DMYDate.today()
print(d)
print(e)

1967-04-09
4/9/1967
9/4/1967
5/5/2025
5/5/2025


The common naming convention for class methods is to influde the word *from_* as prefix. 
One caution about class methods is that python does not manage them in a namespace separate from the isntance methods meaning they can still be invoked on an instance. 


In [6]:
print(a.today())

2025-05-05


### static methods
A static method does not take an extra self or cls arguments and uses a class as merely a namespace for it. You don't normally create instances of such a class but call the functions directly through the class. 